In [1]:
import numpy as np
from keras.models import Sequential, Model
from Model import PolicyGradientModel
from keras.layers import TimeDistributed, Dense, Reshape, Flatten, GRU, Input, Embedding
from keras.optimizers import Adam
from keras.layers.convolutional import Convolution2D

Using TensorFlow backend.


In [2]:
ALLOWED_PARAMETERS = list('XY')
ALLOWED_SYMBOLS = ALLOWED_PARAMETERS + list('0123456789+-*/#')
NUM_SYMBOLS = len(ALLOWED_SYMBOLS)
MAX_LENGTH = 30
CORRECT_EXPRESSION = "3*X+2*Y"

In [3]:
def getModel():
    # Trying to neglect input
    input1 = Input(shape=(1,1))
    # TODO: Add noise layer to make output vary
    x = GRU(32)(input1)
    #x = Flatten()(x)
    out = Dense(NUM_SYMBOLS, activation='sigmoid')(x)


    model = Model(inputs=input1, outputs=out)
    model.compile(optimizer=Adam(),
                loss='categorical_crossentropy',
                metrics=['acc'])
    return model

## TRAINING

In [4]:
model = getModel()
pgModel = PolicyGradientModel(model=model,
                              allowedParam=ALLOWED_PARAMETERS,
                              allowedSymbol=ALLOWED_SYMBOLS,
                              numSymbol=NUM_SYMBOLS,
                              maxLength=MAX_LENGTH,
                              correctExpression=CORRECT_EXPRESSION,
                              learningRate=0.1)

In [5]:
pgModel.train(input=np.ones((1,1,1)))

0 -150 -36
XXX88X9X7*11
0 -150 0

0 -150 -9
21X
0 -150 -27
642932Y83
0 -150 -6
*8
0 -150 -6
Y+
0 -150 -18
6157XX
0 -150 0

-11.0001312381 150 -3
6
0 -150 -90
7444/6158341171935+-7127X/**04
-11.6105057933 150 -12
6-56
0 -150 -51
1/8092/924X95X494
0 -150 -12
01-Y
-13.2377435284 150 -12
-551
0 -150 -81
529X0615**9*-3593745+/78*Y7
0 -150 -42
+31*2761+*9-4+
0 -150 -15
X5*53
0 -150 -42
04063-21Y4Y14Y
0 -150 -48
00/*719X1YX*95X6
-10.9238499699 150 -9
8+3
0 -150 -54
-6-357+61*323X6-93
0 -150 -90
0421436Y+7950Y**57+90X3-05X93/
0 -150 -12
+/35
0 -150 -84
6933-+++3X*57+4/53+627/89607
0 -150 -9
-0X
0 -150 0

0 -150 -45
/6-+063/5*9-Y6*
0 -150 -90
1697X030*15/29-43/+X82086X77XY
0 -150 -54
15299Y8747/19529*/
0 -150 -90
995-+62*75318/6Y*/537189Y53776
0 -150 -90
186X+2X0+099-82222Y26*+29X-942
0 -150 -90
678Y062+5X7X+0284X/X5/3*30*3-9
-12.802571251 150 -18
9--467
0 -150 0

0 -150 -39
+9-X27140677-
-10.9548684615 150 -6
+9
0 -150 -90
2+55+7-+63Y33+7268-//7+21X9/85
-10.9548684615 150 -3
9
0 -150 -48
4*X95

0 -150 -90
7*/8102+-+212X1826-X64X4900Y05
0 -150 -90
*Y9X/4*/61YX-13X-4+X-YY0**98-1
0 -150 -57
49655227-Y3Y9989337
0 -150 -90
9-248+4702/46593X83+408XYY818*
0 -150 -12
168-
0 -150 -90
X74-051158019-2**27/004/779-0/
0 -150 -54
42/62X5*379046--62
0 -150 -90
5795144Y98Y33-3X7X++4Y5847*562
0 -150 -57
6--3Y-578*00Y-7*/55
0 -150 -90
8729+28546X6/1200Y51-52-*+8X*4
0 -150 -30
187+66Y443
0 -150 -27
7*409X--/
0 -150 -51
XX1-5471*+9-7072X
0 -150 -90
92-511+45/-YY66Y033926979/5003
-20.3342059739 150 -27
+X+753051
0 -150 -33
6017398Y0+/
-11.0438494054 150 -3
3
0 -150 -90
25650290Y2-X-*45-99/5845-6-452
0 -150 -90
+*-X4--*-68*/5403094075-*1*6Y1
0 -150 -90
X*9+-0158-+2+*9/13X10+1884+1Y2
0 -150 -90
354/7X7+3197608-2Y1171Y076Y-60
0 -150 -48
10/23+7X32432-17
0 -150 -51
YX411010//*14-82X
-19.2050267096 150 -24
3-243372
0 -150 -33
10785*54X87
0 -150 -75
52042635190XX956Y96-Y/726
0 -150 -81
-7/9583/3/869203XX*44/0X222
0 -150 -15
-+11X
0 -150 -90
80/96++--77YX93150+XY7*-*/X2-9
-11.0996801695 150 -9
3-4
0 -15

0 -150 -51
X32130654120X6364
0 -150 -66
4839*58//80894/7+*0451
0 -150 0

0 -150 -45
259X*71131731-9
0 -150 -51
+6238Y7X928*/0029
0 -150 -90
265Y*308435855Y1-X-48-X832+/27
0 -150 -84
0093688/4-1Y09Y31872574+3772
0 -150 -90
+3Y81720+218217*718X86*7+Y9X86
0 -150 -36
YX8910485164
0 -150 -21
180-+0X
0 -150 -33
9105X/9-792
-10.8628377739 150 -3
Y
-17.9374873104 150 -18
99*693
0 -150 -24
38X/*/99
0 -150 -90
01*701-X18437350Y6//+*6X186369
0 -150 -90
6*6*993650+9+/52149Y086*70+626
0 -150 -87
9913X1963*Y2Y942-8-Y907-/70Y2
0 -150 -90
*59X2YY719292006*66042XX+682/0
0 -150 -60
96135X+/909+/50*128X
0 -150 -81
713/X60X76Y6Y778*038*Y165**
0 -150 -12
X77*
0 -150 -48
52+368**-09-++53
0 -150 -90
61+29Y*Y5/9*/Y64*X/X1/7*Y38*5/
0 -150 -90
X8983137+//*38+3352-27314X/45*
0 -150 -90
5X+X/*XY-*X*Y8118X25883107036/
0 -150 0

-18.1613586972 150 -21
773*111
0 -150 -90
*16-783Y3X-/5*51*-+491102*930*
0 -150 -90
X43961-YX0098**491720**7/*74X-
0 -150 -21
X*/1/Y7
0 -150 -90
X4+/9707X870-59146*527*75Y+176
0 -150 -27
X5

0 -150 -90
87X917X*770//+95-Y6Y450146*23+
0 -150 -90
493-*0+658+XY49914X6X/6674-/Y/
0 -150 -90
*YY7366Y627*28061Y7077608X9*X5
0 -150 -90
7-/011+770X33**99+1Y+9X692532Y
0 -150 -90
+/-9+201/9+5213*2*XX*517++8267
0 -150 -90
X/6444/10*+398/1Y3408X37455XY4
0 -150 -39
+1/*2-06++83-
0 -150 -30
*4X103190-
0 -150 -90
+8446-5442+Y-7*1+*68829510*/-8
0 -150 -90
+54/32047X75Y7Y760+91*5X1415/1
0 -150 -90
Y1/+781*3/3Y8425X6XY4*36258480
0 -150 -90
98*61/12-09X2X/6-2/+Y44**713X7
0 -150 -30
*181126-07
0 -150 -90
5X5390X29X34*7+3*0-59+9-147-0+
0 -150 -87
36--5*/+*584+++*/0*41/6807-**
0 -150 -39
XX18/71635595
0 -150 -60
/79-564-93097*X0962-
0 -150 -90
2608X87808X329+54+72240//5X-8/
0 -150 -90
Y/134-99X4Y6X3686-94--78431818
0 -150 -90
585Y*X8X91791+38053-045Y7601-*
0 -150 -90
XY29/X5582-71684752440/6-95*99
0 -150 -90
4Y67-2+26831+8+0+39Y*3-1Y8/9X9
0 -150 -90
+57274+23X--X*+83-X33*8+536-9Y
-11.0850930306 150 -12
2/31
-18.1492243208 150 -15
84769
0 -150 -45
700-/8+/2253645
0 -150 -90
78903+1404YY63/6X70189*+

0 -150 -90
773069X8+/00Y7619210*0Y28X7284
0 -150 -90
X3X2*3+711707*/8X115938Y-06572
0 -150 -90
X+3X4+104X78255*654912222YY922
0 -150 -39
767*9-8/Y571X
-50.3618677267 150 -90
5984640+776273*253731841370*42
0 -150 -24
62*+/76*
0 -150 -63
8X1302X02Y8*3410*680X
0 -150 -69
354605+3-809X/248/72520
0 -150 -90
+6423++-Y-4+2248/1/267+++6+/2*
0 -150 -90
7*+3385*884*2++4616X22+793+535
0 -150 -15
8Y/*0
-10.3148684611 150 -6
99
0 -150 -90
46531349+4769+-42933+9509X1347
-76.4299938901 150 -48
25/3+52/X-7634/5
0 -150 -90
*779Y-3525Y5235*-++1//1-+3*703
0 -150 -24
0*41033*
0 -150 -90
/*8983/43/1/2Y371282664610/1+-
0 -150 -90
9+8X8*83--++78+92803*0+8285*X/
0 -150 -72
//-64121Y-54*5/284*70742
0 -150 -90
4Y01791+5/74*17Y*84117+45X--6+
0 -150 -39
YY+*4520116-1
0 -150 -90
014/55/46586359663379/733511*7
0 -150 -21
/1X4745
0 -150 -90
42880X8+9786064-2529X21X93/4--
0 -150 -90
2X63+883963Y209833391**X35744Y
0 -150 -90
+09+1/6079*7+069+608203//3156-
0 -150 -90
1879+/Y99+8017-X16+X-36385+134
0 -150 -87
+2*7-68-78

...

0 -150 -90
Y2/41914-5835391-752+682-6/224
0 -150 -90
23310930+96158X/60063Y1-979527
-34.8342941492 150 -90
154491232272-5643//1*291977092
0 -150 -90
0/5X-5981722++781447/6-54X7184
0 -150 -90
5**74Y0944330/9X60+6X3+X3*5+-4
-11.0580705129 150 -3
2
0 -150 -90
*7/5/8/3-32+Y277*7-57079*3X17Y
0 -150 -90
97039/4740-X180407470-1Y0258-0
0 -150 -90
X9/59+932+937006/-2584*39X+0/5
0 -150 -81
*69X95420999352631554776975
0 -150 -90
3534--3+5552+-1244105*33-65/2Y
0 -150 -90
7892Y7-4667910*93948769931467X
0 -150 -90
Y-2799489658+*54Y*Y691*37614*2
0 -150 -90
6Y5/4/0/587568193/490945-13167
-37.0186059609 150 -63
5011*2647337944-1/371
0 -150 -90
4581Y75/22/5-6058-3284829926+4
0 -150 -63
5841335879/5-+9Y3+/37
0 -150 -81
3/7315+9376X02/19+761605447
0 -150 -90
0527*9X3950/337835+2X4191-4-71
0 -150 -90
73491294+8+-0771/5852+8X561432
0 -150 -90
1/07*5506440X6Y0*92961Y55834X4
-12.1330239959 150 -90
11*312837644*+6119/75453995368
0 -150 -90
*49+48*501/23+2211+1553-585Y5X
0 -150 -90
5/37*24141+979-36/121+19*6946

0 -150 -90
66814/12237-1+61-0*55X0379*394
-37.6184769356 150 -90
17-431174993-24168332842956+29
-19.8425316098 150 -42
4*5-+3223354/7
0 -150 -90
7-76760X84+X+7564XX00421633974
0 -150 -90
3437522Y495959506611/090514428
0 -150 -81
XY70Y1597Y947618196748*1628
0 -150 -90
++398993*94-39+5-7-90+93/+5+/3
0 -150 -69
03365YX641-+57Y-4-0+433
0 -150 -90
14277+97199Y9766++74039-9436-6
0 -150 -90
35-Y+525177725Y23+66/4892291*2
0 -150 -12
/987
-28.2910095872 150 -90
2172*2451*+916-7026134652-1704
0 -150 -63
-798297X31-3959635427
0 -150 -90
-0-43+32901268625+131-2639891/
0 -150 -90
4-05238*711//74+/0544151+63936
0 -150 -90
0161534-/4*39X*X27/3-2+71+/49-
0 -150 -90
9404*4*7+05*234311*5-41/*/+146
0 -150 -90
57*464*430545-Y/9-*6952*1-X018
-29.0267533782 150 -90
-8973178331/2+7/534548/24/4134
0 -150 -90
+55X05103-3X84749811706+041*71
0 -150 -90
5-1649598+0/569-*8/5876749--94
0 -150 -90
65Y333-+2Y21X/71Y7776248912781
0 -150 -90
6717*263547860Y*19-180108792-4
0 -150 -90
97*9X+09Y1236-Y4+7259469750092
0 -15

0 -150 -90
337*68507467Y679191749*80+2423
0 -150 -90
14754YX145542+264-5/3Y92--0630
0 -150 -30
970761Y603
0 -150 -90
/64/9291+4Y-120X900720077-9/17
-20.793620353 150 -90
1/6*7181037---7714+29913/10+54
0 -150 -90
5932-9*-99+-5449-X/3332631250-
0 -150 -90
1716-39*58754*419933X*433233/3
0 -150 -90
-4*997Y/19X9992/9927+2299+767-
0 -150 -90
-441X33X3Y9597-13546-3Y9213X3+
0 -150 -90
4+4799019913469237816-3X392149
0 -150 -90
-58536-X0*5762*6129+1648177-30
-32.7308295232 150 -90
-2*963117-6*37960359090*4/5+49
0 -150 -90
45-/-655/591+9-0989/15919X2094
0 -150 -90
1/658*2353471+/771*93933+607-9
0 -150 -90
*7747361/4242622-70*694+9153/4
0 -150 -90
-397-52-67*9Y7*Y4/527580429546
0 -150 -90
917305+761715-4-449X1*447+/5-7
0 -150 -90
65*4028*33XX3+71+930-100X1/*33
-20.0240594363 150 -30
551680+592
-42.6672887354 150 -90
7476863*503748399-733*89397109
0 -150 -90
*34-054X*061574+977017*31790*0
-43.9170262487 150 -90
78/37-437210592967*36*835+4-+5
0 -150 -90
4977+-8528*790/92771-X2447229*
0 -150 -90
3632

0 -150 -90
632724209155Y69979050445089210
0 -150 -90
83//5/97-2-63-33750+3Y348705+9
0 -150 -90
795979-115429+398918070/*4Y79-
0 -150 -90
1158X584-300026+X885Y3316/6*/-
0 -150 -90
*183*-*5494989Y5274913X8519687
0 -150 -90
160-81+19*/*-*5/676835*07//141
0 -150 -90
7+1X43142970*29+4121+5063915*1
-15.7077156019 150 -90
-3-539/6478-7-837777386/114945
0 -150 -90
+01*771*556X47441228124X+8494+
0 -150 -90
/-13959-453114154458442/637246
-20.4389646429 150 -90
838067+95-36+349-6+6-3836+1589
0 -150 -90
13*7Y12354442X6-91212/-+9916+8
0 -150 -90
10/418Y47925549980-8-2791*1157
0 -150 -54
4*+2431128036X455*
0 -150 -90
3040257407XY1-31-*5777505690+5
0 -150 -90
3+1225031/4/-965951741*1907Y4/
0 -150 -90
69587*926307+Y8765X00+6430X*6X
0 -150 -90
545793196909681202+8474//6+6-/
0 -150 -90
5+14Y1X45015--7426083434559172
0 -150 -90
3520-3570295+97-6Y6398-18+-60*
0 -150 -90
5*X194-31*3333325616*063723737
0 -150 -90
865373-914564/336-+57316467-*6
-16.0937311427 150 -90
6-1976*223141*74*87/5/52679631
-38.753638

KeyboardInterrupt: 